In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!unzip /kaggle/input/spooky-author-identification/train.zip
!unzip /kaggle/input/spooky-author-identification/test.zip
!unzip /kaggle/input/spooky-author-identification/sample_submission.zip

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import numpy as np
from nltk.corpus import stopwords
from tqdm import tqdm
from string import punctuation

In [ ]:
corpus = list(train['text'].values)
sw = stopwords.words('english')
vocab1 = list()
vocab = list()
for i in corpus:
    for j in punctuation:
        i = i.replace(j,'')
    vocab1.append(i)
vocab1 = [i.split() for i in vocab1]
for i in range(len(vocab1)):
    for k in sw:
        temp = [j for j in vocab1[i] if j not in sw]
    vocab.append(temp)


In [ ]:
z = [len(i) for i in vocab]
max_len = np.max(z)
max_words = 15000
max_len

In [ ]:
tk = Tokenizer(num_words=max_words)
tk.fit_on_texts(vocab)
train_encoded = tk.texts_to_sequences(vocab)
test_encoded = tk.texts_to_sequences(test['text'].values)
x_train = pad_sequences(train_encoded, max_len)
x_test = pad_sequences(test_encoded, max_len)

In [ ]:
print(vocab[0],'\n', train_encoded[0],'\n', tk.word_index['however'])

In [ ]:
labels_train = pd.get_dummies(train['author'])
y_train = []
for i in range(len(labels_train)):
    temp = [labels_train['EAP'][i], labels_train['HPL'][i], labels_train['MWS'][i]]
    y_train.append(temp)

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 512, input_length=max_len))
model.add(LSTM(128))
model.add(Dropout(0.7))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint('weights.hdf5',save_best_only=True, monitor='val_loss', mode='auto', verbose=1)
earlystop = EarlyStopping(monitor='val_loss', mode='auto', patience=20)
model.fit(x_train, y_train, epochs=4, batch_size=100, callbacks=[checkpoint,earlystop], verbose=1, validation_split=0.1)

In [ ]:
import os
os.remove('sample_submission.csv')
os.remove('train.csv')
os.remove('test.csv')

In [ ]:
model.load_weights('weights.hdf5')

In [ ]:
pred = model.predict(x_test)

In [ ]:
id = test['id'].values
eap = pred[:,0]
hpl = pred[:,1]
mws = pred[:,2]
submission = {'id':id, 'EAP':eap, 'HPL':hpl, 'MWS':mws}
submission = pd.DataFrame(submission)
submission.to_csv('Submission.csv',index=False)